In [1]:

#max 27
#sonra devam et
#env değiştirildi. 
#best snakes so far tmp/Actor_v2.0-copy1-savepoint3 tmp/Critic_v2.0-copy1-savepoint3

In [2]:
import numpy as np
import import_ipynb
from envwithseen0gamma import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

env=environement()
info={}
env.env_init(info)

class a2c_v2:
    def __init__(self, info={}):
        self.n_actions = info.get("n_actions",4) 
        self.input_shape=info.get("input_shape",(10,10,2)) 
        self.EPISODES = info.get("episodes",100000) 
        self.lr = info.get("lr",0.001) 
        self.gamma=info.get("gamma",0.99)
        self.Actor, self.Critic = self.create_model(input_shape=self.input_shape, n_actions=self.n_actions, lr=self.lr)
        self.states, self.actions, self.rewards = [], [], []
        
    def create_model(self,input_shape,n_actions,lr):
        inputs= keras.Input(shape=input_shape)
        x = layers.Conv2D(32, 4, activation="relu",padding="same")(inputs)
        x=layers.MaxPooling2D(2)(x)
        x=layers.Conv2D(128, 2, activation="relu")(x)
        x=layers.MaxPooling2D(2)(x)
        x=layers.Flatten()(x)
        x=layers.Dense(256,activation="relu")(x)
        action = layers.Dense(n_actions, activation="softmax")(x)
        value = layers.Dense(1)(x)

        Actor = keras.Model(inputs = inputs, outputs = action)
        Actor.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=lr))

        Critic = keras.Model(inputs = inputs, outputs = value)
        Critic.compile(loss='mse', optimizer=keras.optimizers.RMSprop(lr=lr))

        return Actor, Critic
    def remember(self, state, action, reward):
        self.states.append(np.reshape(state,(1,10,10,2)))
        action_onehot = np.zeros([self.n_actions])
        action_onehot[action] = 1
        self.actions.append(action_onehot)
        self.rewards.append(reward)
    def act(self, state):
        prediction = self.Actor.predict(np.reshape(state,(1,10,10,2)))[0]
        #print(prediction)
        action = np.random.choice(self.n_actions, p=prediction)
        return action
    def discount_rewards(self, reward,states):
        gamma=0.01*env.length    # discount rate
        running_add = 0
        discounted_r = np.zeros_like(reward,dtype=float)
        for i in reversed(range(0,len(reward))):
            running_add = running_add * gamma + reward[i]
            discounted_r[i] = running_add
        return discounted_r
        
    def replay(self):
        states = np.vstack(self.states)
        actions = np.vstack(self.actions)
        discounted_r = self.discount_rewards(self.rewards,states)
        values = self.Critic.predict(states)[:, 0]
        advantages = discounted_r - values
        # training Actor and Critic networks
        self.Actor.fit(states, actions, sample_weight=advantages, epochs=1, verbose=0)
        self.Critic.fit(states, discounted_r, epochs=1, verbose=0)
        self.states, self.actions, self.rewards = [], [], []
    def step(self, action):
        reward, next_state, done,score = env.step(action)
        return next_state, reward, done,score
    def run(self):
        for e in range(self.EPISODES):
            state = env.start()
            done = False
            i=0
            while not done:
                i+=1
                action = self.act(state)
                next_state, reward, done,score= self.step(action)
                #print(state[:,:,0])
                self.remember(state, action, reward)
                state = next_state
                if done:
                    print("Episode :",e," Score :",score,"length :" , i)
                    if(e%1000==0):
                        self.save()
                    self.replay()
    def save(self):
        self.Actor.save('tmp/Actorway')
        self.Critic.save('tmp/Criticway')
    def load(self):
        self.Actor = tf.keras.models.load_model('tmp/Actorway')
        self.Critic = tf.keras.models.load_model('tmp/Criticway')


importing Jupyter notebook from envwithseen0gamma.ipynb
False
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
-0.1


In [3]:
a=a2c_v2()
a.Critic.summary()
a.load()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 10, 2)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 10, 10, 32)        1056      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 128)         16512     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)              

In [ ]:
a.run()

Episode : 0  Score : 8 length : 60
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: tmp/Actorway\assets
INFO:tensorflow:Assets written to: tmp/Criticway\assets
Episode : 1  Score : 8 length : 72
Episode : 2  Score : 8 length : 71
Episode : 3  Score : 16 length : 143
Episode : 4  Score : 1 length : 12
Episode : 5  Score : 12 length : 112
Episode : 6  Score : 5 length : 51
Episode : 7  Score : 11 length : 91
Episode : 8  Score : 9 length : 74
Episode : 9  Score : 11 length : 81
Episode : 10  Score : 13 length : 126
Episode : 11  Score : 18 length : 189
Episode : 12  Score : 10 length : 77
Episode : 13  Score : 12 length : 123
Episode : 14  Score : 4 length : 38
Episode : 15  Score : 12 length : 83
Episode : 16  Score : 9 length : 82
Episode : 17  Score : 8 length : 86
Epi

In [ ]:
a.save()